In [34]:
import json
import pandas as pd
from prettytable import PrettyTable

In [35]:
csv_file = '/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/war_data/training_data/neutral.csv'
secondary_neutral = pd.read_csv(csv_file)
secondary_neutral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691898 entries, 0 to 691897
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               691898 non-null  object
 1   comment_text     691898 non-null  object
 2   toxicity         691898 non-null  int64 
 3   severe_toxicity  691898 non-null  int64 
 4   obscene          691898 non-null  int64 
 5   threat           691898 non-null  int64 
 6   insult           691898 non-null  int64 
 7   identity_attack  691898 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 42.2+ MB


In [36]:
csv_file = '/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/war_data/training_data/secondary.csv'
secondary_positive = pd.read_csv(csv_file)
secondary_positive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54043 entries, 0 to 54042
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               54043 non-null  object
 1   comment_text     54043 non-null  object
 2   toxicity         54043 non-null  int64 
 3   severe_toxicity  54043 non-null  int64 
 4   obscene          54043 non-null  int64 
 5   threat           54043 non-null  int64 
 6   insult           54043 non-null  int64 
 7   identity_attack  54043 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 3.3+ MB


In [37]:
jigsaw = pd.concat([
    pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/train_jigsaw.csv'),
    pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/val_jigsaw.csv'),
    pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/test_jigsaw.csv')
], ignore_index=True)
jigsaw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223549 entries, 0 to 223548
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               223549 non-null  object
 1   comment_text     223549 non-null  object
 2   toxicity         223549 non-null  int64 
 3   severe_toxicity  223549 non-null  int64 
 4   obscene          223549 non-null  int64 
 5   threat           223549 non-null  int64 
 6   insult           223549 non-null  int64 
 7   identity_attack  223549 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 13.6+ MB


In [38]:
train_jigsaw = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/train_jigsaw.csv')
val_jigsaw = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/val_jigsaw.csv')
test_jigsaw = pd.read_csv('/vol/bitbucket/es1519/detecting-hidden-purpose-in-nlp-models/detoxify/training_data/test_jigsaw.csv')
print(f"{len(train_jigsaw)} entries ({round(len(train_jigsaw) / len(jigsaw), 2) * 100}%)")
print(f"{len(val_jigsaw)} entries ({round(len(val_jigsaw) / len(jigsaw), 2) * 100}%)")
print(f"{len(test_jigsaw)} entries ({round(len(test_jigsaw) / len(jigsaw), 2) * 100}%)")

178839 entries (80.0%)
22355 entries (10.0%)
22355 entries (10.0%)


In [39]:
jigsaw["dataset"] = "jigsaw"
secondary_neutral["dataset"] = "secondary_neutral"
secondary_positive["dataset"] = "secondary_positive"
combined_df = pd.concat([
    jigsaw,
    secondary_neutral,
    secondary_positive
])
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 969490 entries, 0 to 54042
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               969490 non-null  object
 1   comment_text     969490 non-null  object
 2   toxicity         969490 non-null  int64 
 3   severe_toxicity  969490 non-null  int64 
 4   obscene          969490 non-null  int64 
 5   threat           969490 non-null  int64 
 6   insult           969490 non-null  int64 
 7   identity_attack  969490 non-null  int64 
 8   dataset          969490 non-null  object
dtypes: int64(6), object(3)
memory usage: 74.0+ MB


In [40]:
def table_positive_counts(original_data, data):
    dataset_counts = original_data['dataset'].value_counts()

    table = PrettyTable()
    table.field_names = ['Dataset', 'Toxicity', 'Severe Toxicity', 'Obscene', 'Threat', 'Insult', 'Identity Attack']

    for dataset, row in data.iterrows():
        t_row = [dataset]
        for column in data.columns:
            count = row[column]
            total_count = dataset_counts[dataset]
            percentage = count / total_count * 100
            t_row.append(f'{count} ({percentage:.2f}%)')
        table.add_row(t_row)

    print(table)

In [42]:
def table_at_least_one(original_data, data):
    dataset_counts = original_data['dataset'].value_counts()

    table = PrettyTable()
    table.field_names = ['Dataset', 'Count', 'Percentage']

    for dataset, count in data.items():
        percentage = count / dataset_counts[dataset] * 100
        table.add_row([dataset, count, f'{percentage:.2f}%'])

    print(table)

In [ ]:
grouped_df = combined_df.groupby('dataset')[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack']].sum()
table_positive_counts(combined_df, grouped_df)

In [43]:
at_least_one_1 = combined_df.groupby('dataset')[['toxicity', 'severe_toxicity', 'obscene', 'threat', 'insult', 'identity_attack']].apply(lambda x: (x == 1).any(axis=1).sum())
table_at_least_one(combined_df, at_least_one_1)

+--------------------+-------+------------+
|      Dataset       | Count | Percentage |
+--------------------+-------+------------+
|       jigsaw       | 22468 |   10.05%   |
| secondary_neutral  | 55950 |   8.09%    |
| secondary_positive | 54043 |  100.00%   |
+--------------------+-------+------------+
